# Начало работы

LCEL облегчает создание сложных цепочек на основе базовых компонентов и по умолчанию поддерживает такие функции как потоковая передача, параллелизм и журналирование.

:::note

Для работы с GigaChat вам понадобится получить авторизационные данные.
Подробную инструкцию о том как это сделать вы найдете в официальной документации:

* [Получение авторизационных данных для физических лиц](https://developers.sber.ru/docs/ru/gigachat/individuals-quickstart#shag-1-sozdayte-proekt-giga-chat-api)
* [Получение авторизационных данных для ИП и юридических лиц](https://developers.sber.ru/docs/ru/gigachat/legal-quickstart#shag-3-poluchite-avtorizatsionnye-dannye)

:::

## Базовый пример: промпт + модель + парсер вывода {#basic-example}

Наиболее распространенный сценарий — объединение в цепочку шаблона промпта и модели.
Посмотрим, как это работает на примере цепочки, которая генерирует шутку на заданную тему.

In [ ]:
%pip install --upgrade --quiet  gigachain

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

In [ ]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [2]:
from langchain.chat_models.gigachat import GigaChat
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Расскажи шутку про {topic}")
model = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "облако"})

'Почему облака не любят понедельники? Потому что они всегда идут после субботы и воскресенья.'

Обратите внимание на строку, в которой различные компоненты объединяются в цепочку с помощью LCEL:

```
chain = prompt | model | output_parser
```

Символ `|` выполняет ту же функцию, что и [оператор конвейера в Unix](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D0%B2%D0%B5%D0%B9%D0%B5%D1%80_(Unix)) — связывает вместе различные компоненты и подает вывод одного компонента на вход в следующий компонент.

В этой цепочке данные, введенные пользователем, передаются в шаблон промпта, затем вывод шаблона передается на вход в модель, а затем результат работы модели передается в парсер выходных данных.
Рассмотрим каждый компонент в отдельности.

### 1. Промпт

Объект `prompt` - экземпляр `BasePromptTemplate`.
Он принимает на вход словарь с переменными шаблона и возвращает `PromptValue`.
`PromptValue` - это обертка вокруг готового промпта, которую можно передать в `LLM` (принимает строку в качестве входных данных) или в `ChatModel` (принимает на вход последовательность сообщений).
`PromptValue` поддерживает разные типы языковых моделей, поскольку содержит логику как для создания экземпляров `BaseMessage`, так и для создания строк.

In [3]:
prompt_value = prompt.invoke({"topic": "облако"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='Расскажи шутку про облако')])

In [4]:
prompt_value.to_messages()

[HumanMessage(content='Расскажи шутку про облако')]

In [5]:
prompt_value.to_string()

'Human: Расскажи шутку про облако'

### 2. Модель

Готовый экземпляр `PromptValue` передается в `model`.
В примере в качестве модели (`model`) используется класс `ChatModel`, который возвращает экземпляр класса `BaseMessage`.

In [6]:
message = model.invoke(prompt_value)
message

AIMessage(content='Почему облака не любят понедельники? Потому что они всегда идут после субботы и воскресенья.')

Если бы в `model` использовался класс `LLM`, то на выходе была бы строка.

In [7]:
from langchain_openai import OpenAI

llm = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)
llm.invoke(prompt_value)

'Почему облако не может перейти дорогу?\n\nОтвет: Потому что оно белое и пушистое!'

### 3. Парсер вывода

И, наконец, выходные данные из модельи `model` передаются в `output_parser` — экземпляр класса `BaseOutputParser`, который принимает на вход строку или экземпляр `BaseMessage`.
Объект `StrOutputParser` отвечает за преобразование любых входных данных в строку.

In [8]:
output_parser.invoke(message)

'Почему облака не любят понедельники? Потому что они всегда идут после субботы и воскресенья.'

### 4. Итоговый процесс

Получившуюся цепочку можно описать шагами:

1. Ввод пользователя передается как `{"topic": "облако"}`.
2. Компонент `prompt` принимает данные от пользователя, которые, после создания промпта с помощью темы `topic`, используются для создания `PromptValue`.
3. Компонент `model` принимает сгенерированный промпт и передает его в GigaChat. Модель возвращает результат генераци в виде экземпляра `ChatMessage`.
4. Компонент `output_parser` принимает `ChatMessage`, и преобразует это в Python-строку, которая возвращается из метода `invoke`.

```mermaid
graph LR
    A(Input: topic=Облако) --> |Dict| B(PromptTemplate)
    B -->|PromptValue| C(ChatModel)    
    C -->|ChatMessage| D(StrOutputParser)
    D --> |String| F(Результат)
```


:::note

Для просмотра выходных данных любого из компонентов, простестируйте сокращенную версию цепочки, например, `prompt` или `prompt | model`.

:::

In [ ]:
input = {"topic": "облако"}

prompt.invoke(input)
# > ChatPromptValue(messages=[HumanMessage(content='Расскажи шутку про облако')])

(prompt | model).invoke(input)
# > AIMessage(content='Почему облака не любят понедельники? Потому что они всегда идут после субботы и воскресенья.')

## Пример поиска с RAG

Следующий пример демонстрирует цепочку с RAG (retrieval-augmented generation), с помощью которой можно добавить контекст при ответе на вопросы.

In [5]:
# Для работы примера установите docarray tiktoken:
# pip install langchain docarray tiktoken

from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_community.vectorstores.docarray import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

vectorstore = DocArrayInMemorySearch.from_texts(
    ["Василий работал в Сбере", "медведи любят есть мед"],
    embedding=GigaChatEmbeddings(
        credentials="<авторизационные_данные>", verify_ssl_certs=False
    ),
)
retriever = vectorstore.as_retriever()

template = """Отвечай на вопрос только на основе контекста:
{context}

Вопрос: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("где работал Василий?")

'Василий работал в Сбере.'

Цепочка в примере выглядит так:

In [6]:
chain = setup_and_retrieval | prompt | model | output_parser

Чтобы понять как работает цепочка, обратите внимание, что шаблон промпта содержит переменные `question` (вопрос пользователя) `context` (контекст вопроса), значения которых используются в промпте.
Перед созданием промпта из `vectorstore` нужно получить документы, которые модель будет использовать в качестве контекста при ответе на вопрос.

Для этого, на этапе предварительной подготовки цепочки, добавлен ретривер, который на основе запроса извлекает документы из векторного хранилища `vectorstore`.
Ретривер — исполняемый компонент, который можно объединять с другими компонентами или запускать отдельно:



In [7]:
retriever.invoke("где работал Василий?")

[Document(page_content='Василий работал в Сбере'),
 Document(page_content='медведи любят есть мед')]

После этого, с помощью `RunnableParallel` на основе полученных документов и вопроса пользователя подготавливается промпт, который будет передан в модель.
При этом для поиска документов используется ретривер, для получения вопроса пользователя `RunnablePassthrough`:

In [8]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

Готовая цепочка будет выглядеть так:

In [9]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

Этапы работы цепочки:

1. Вначале создается экземпляр `RunnableParallel` с двумя записями: вопросом и контекстом. Запись `context` содержит документы для поиска по контексту, которые получил ретривер. Запись `question` содержит вопрос пользователя. Для передачи вопроса использвуется экземпляр `RunnablePassthrough`, с помощью которого копируется запись.
2. Словарь с записями, созданный на предыдщуем этапе, передается в компонент `prompt`. Компонент принимает вопрос пользователя (`question`) и контекст (`context`), извлеченный из документа, использует полученные данные для создания промпта и возвращает `PromptValue`.
3. Компонент `model` принимает сгенерированный промпт и передает его в GigaChat. Модель возвращает результат генераци в виде экземпляра `ChatMessage`.
4. Компонент `output_parser` принимает `ChatMessage`, и преобразует это в Python-строку, которая возвращается из метода `invoke`.

```mermaid
graph LR
    A(Вопрос) --> B(RunnableParallel)
    B -->|Вопрос| C(Ретривер)
    B -->|Вопрос| D(RunnablePassThrough)
    C -->|context=извлеченные документы| E(PromptTemplate)
    D -->|question=Djghjc| E
    E -->|PromptValue| F(ChatModel)    
    F -->|ChatMessage| G(StrOutputParser)
    G --> |String| H(Ответ модели)
```

## Смотрите также

* [Получение авторизационных данных для физических лиц](https://developers.sber.ru/docs/ru/gigachat/individuals-quickstart#shag-1-sozdayte-proekt-giga-chat-api)
* [Получение авторизационных данных для ИП и юридических лиц](https://developers.sber.ru/docs/ru/gigachat/legal-quickstart#shag-3-poluchite-avtorizatsionnye-dannye)